In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from tqdm import tqdm
import json
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
pd.options.plotting.backend = "plotly"

https://disk.yandex.ru/d/b5NdFMfnDEVyTQ

In [3]:
df = pd.read_csv('sub_final_6.csv', index_col='Unnamed: 0')
df = df.drop('trans_day', axis=1)

In [4]:
df

,amount,target,mcc_742,mcc_1711,mcc_1731,mcc_1799,mcc_2741,mcc_3000,mcc_3351,mcc_3501,...,city_1,city_2,city_3,city_4,city_5,city_6,city_7,city_8,city_9,city_10
d1bbbc9a0e0410d3cf12a3d2f44f3450,"[-1808.56, -3390.41, -144.5, -3542.3, -3542.70...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
81c7349f7bdcb661bc7d5f5f0fdac6aa,"[7954.51, -216.74, -252.19000000000003, -362.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
fc0c7c937f139bd6d1c5b5d0a0023dd3,"[-12293.56, 10484.57, 21693.390000000003, 1084...",NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3495487d5beaa5caaa0682fddc38379a,"[49.52, -1445.24, 506.98, -6.0, 6901.08, 181.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07e279042b1da23710fa2f65034d30b9,"[45.29, -691.78, 449.09, 329.13, -2182.44, 325...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a517cdee04c890b311fc876a325dba99,"[-36.61, 35.18]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
b6714c0ca16193cbd419623eb70b1488,[-73.31],1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ee4c5d78cbb27771979714ffa903ef5d,[72.71000000000001],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d428ad799c5790b036beb67cc1b81075,"[-14463.42, -18078.07]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
amount_len = []
for el in list(df['amount']):
    amount_len.append(len(el))

In [6]:
mean_p = []
median_p = []
mean_n = []
median_n = []
std_p = []
std_n = []
min_n = []
min_p = []
max_n = []
max_p = []
sum_p = []
sum_n = []
for el in list(df['amount']):
    el = [float(x) for x in el.replace('[', '').replace(']', '').split(',')]
    el = np.array(el)
    el_p = el[el > 0]
    el_n = el[el < 0]
    if len(el_p) == 0:
        el_p = np.array([0])
    if len(el_n) == 0:
        el_n = np.array([0])
    mean_p.append(np.mean(el_p))
    mean_n.append(np.mean(el_n))
    median_p.append(np.median(el_p))
    median_n.append(np.median(el_n))
    std_p.append(np.std(el_p))
    std_n.append(np.std(el_n))
    min_n.append(np.min(el_n))
    min_p.append(np.min(el_p))
    max_n.append(np.max(el_n))
    max_p.append(np.max(el_p))
    sum_p.append(np.sum(el_p))
    sum_n.append(np.sum(el_n))

In [7]:
df['number_of_purch'] = amount_len
df['mean_p'] = mean_p
df['mean_n'] = mean_n
df['median_p'] = median_p
df['median_n'] = median_n
df['std_p'] = std_p
df['std_n'] = std_n
df['min_p'] = min_p
df['min_n'] = min_n
df['max_p'] = max_p
df['max_n'] = max_n
df['sum_p'] = sum_p
df['sum_n'] = sum_n

In [13]:
df = df.dropna()

In [14]:
X = df.drop(['amount', 'target'], axis=1)
y = df['target']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [16]:
model = CatBoostClassifier(iterations=5000, depth=7, learning_rate=0.01, l2_leaf_reg=2)

In [17]:
model.fit(X_train, y_train)

0:	learn: 0.6899850	total: 165ms	remaining: 13m 47s
1:	learn: 0.6867263	total: 185ms	remaining: 7m 42s
2:	learn: 0.6837063	total: 204ms	remaining: 5m 39s
3:	learn: 0.6809834	total: 223ms	remaining: 4m 38s
4:	learn: 0.6781641	total: 242ms	remaining: 4m 1s
5:	learn: 0.6752867	total: 261ms	remaining: 3m 36s
6:	learn: 0.6727371	total: 280ms	remaining: 3m 19s
7:	learn: 0.6703436	total: 299ms	remaining: 3m 6s
8:	learn: 0.6674060	total: 319ms	remaining: 2m 56s
9:	learn: 0.6643226	total: 339ms	remaining: 2m 49s
10:	learn: 0.6620312	total: 358ms	remaining: 2m 42s
11:	learn: 0.6596223	total: 378ms	remaining: 2m 37s
12:	learn: 0.6568289	total: 400ms	remaining: 2m 33s
13:	learn: 0.6546727	total: 423ms	remaining: 2m 30s
14:	learn: 0.6522826	total: 442ms	remaining: 2m 26s
15:	learn: 0.6495940	total: 461ms	remaining: 2m 23s
16:	learn: 0.6476698	total: 480ms	remaining: 2m 20s
17:	learn: 0.6453275	total: 499ms	remaining: 2m 18s
18:	learn: 0.6430898	total: 518ms	remaining: 2m 15s
19:	learn: 0.6410788	to

In [18]:
roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

0.8667738608917472